<a href="https://colab.research.google.com/github/Amt15/Pyspark/blob/main/how_to_avoid_few_lines_from_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark -q
!pip install findspark -q

     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 67.8 MB/s 


In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("Avoiding few lines from CSV")\
        .master("local[*]")\
        .enableHiveSupport()\
        .getOrCreate()

In [24]:
df = spark.read.csv("/content/avoiding_few_lines_from.csv",header=True)
df.show(truncate=False)

+-----+
|line1|
+-----+
|line2|
|line3|
|line4|
|id   |
|1    |
|2    |
|3    |
|4    |
|5    |
+-----+



In [25]:
rdd = spark.sparkContext.textFile("/content/avoiding_few_lines_from.csv")
#rdd.collect()
rdd.zipWithIndex().collect()

[('line1', 0),
 ('line2', 1),
 ('line3', 2),
 ('line4', 3),
 ('id,name,loc', 4),
 ('1,ravi,patna', 5),
 ('2,ram,bangalore', 6),
 ('3,prasad,chennai', 7),
 ('4,sam,pune', 8),
 ('5,shyam,delhi', 9)]

In [26]:
rdd.zipWithIndex().filter(lambda x: x[1]>3).collect()

[('id,name,loc', 4),
 ('1,ravi,patna', 5),
 ('2,ram,bangalore', 6),
 ('3,prasad,chennai', 7),
 ('4,sam,pune', 8),
 ('5,shyam,delhi', 9)]

In [27]:
rdd.zipWithIndex().filter(lambda x: x[1]>3).map(lambda x: x[0]).collect()

['id,name,loc',
 '1,ravi,patna',
 '2,ram,bangalore',
 '3,prasad,chennai',
 '4,sam,pune',
 '5,shyam,delhi']

In [28]:
rdd.zipWithIndex().filter(lambda x: x[1]>3).map(lambda x: x[0].split(",")).collect()

[['id', 'name', 'loc'],
 ['1', 'ravi', 'patna'],
 ['2', 'ram', 'bangalore'],
 ['3', 'prasad', 'chennai'],
 ['4', 'sam', 'pune'],
 ['5', 'shyam', 'delhi']]

In [29]:
rdd_final = rdd.zipWithIndex().filter(lambda x: x[1]>3).map(lambda x: x[0].split(","))

columns = rdd_final.collect()[0]
columns

['id', 'name', 'loc']

In [32]:
skipline = rdd_final.first()
#rdd_final.toDF().show()
df= rdd_final.filter(lambda x : x != skipline).toDF(columns)
df.show()

+---+------+---------+
| id|  name|      loc|
+---+------+---------+
|  1|  ravi|    patna|
|  2|   ram|bangalore|
|  3|prasad|  chennai|
|  4|   sam|     pune|
|  5| shyam|    delhi|
+---+------+---------+

